In [0]:
%sql
SELECT * FROM simulated_blockchain_transactions
LIMIT 10;

tx_id,timestamp,from_address,to_address,amount,is_mixer,entity_type,cluster_id,label
6f7d1f01d5e74b108b9249e50c385331,11/07/2022 23:37,addr_b1d3f097,addr_db93d46e,0.10745528,false,individual,null,licit
ad0b78e2615545b991969e847fb8b3a8,11/09/2022 09:30,addr_6c33deab,addr_b30130b4,0.11586501,false,exchange,cluster_181,licit
f5ade982a233449a967eb1e809742608,13/01/2022 02:15,addr_d3c79689,addr_09509d9b,0.56944013,false,individual,cluster_75,licit
201e7c036e494adeac14f61549bfdcdd,23/07/2022 10:33,addr_6537be66,addr_2fc403c1,0.4655867,false,individual,cluster_2,licit
ae05b047dddf4ef2bc440ce6adcb6ee1,09/03/2022 10:17,addr_35b7220d,addr_1c4b70ac,0.25443045,false,illicit,null,illicit
9cc0fbb4d9434353a40051fce0752b55,23/12/2022 04:56,addr_4af603b1,addr_032fe314,0.38718698,false,exchange,cluster_53,licit
3b2315c76c534b0988f329c0ed6dead1,28/03/2022 05:37,addr_46f394f2,addr_d5e09ae7,0.05122399,false,individual,cluster_199,licit
a100b21ff1e94aeeb71fd2ec3032bdd3,12/08/2022 19:53,addr_0514d98a,addr_263a24b6,1.23422775,false,individual,null,licit
e3dbaf37cd1e47f0a2d4e1174272d93a,26/01/2022 19:33,addr_3edaba77,addr_2548cf7c,0.18206891,false,exchange,cluster_164,licit
1180d346083b44479fb06edfe3c95baf,14/08/2022 15:47,addr_05ead89f,addr_675404b2,0.39964229,false,exchange,null,licit


Total Transactions

In [0]:
%sql
SELECT COUNT(*) AS total_transactions
FROM simulated_blockchain_transactions;

total_transactions
20000


Transactions By Entity Type

In [0]:
%sql
SELECT entity_type, COUNT(*) AS txn_count
FROM simulated_blockchain_transactions
GROUP BY entity_type
ORDER BY txn_count DESC;


entity_type,txn_count
individual,10146
exchange,4799
illicit,1981
marketplace,1976
unknown,1098


Average Transaction Amount by Entity Type

In [0]:
%sql
SELECT entity_type, ROUND(AVG(amount), 6) AS avg_amount
FROM simulated_blockchain_transactions
GROUP BY entity_type;

entity_type,avg_amount
marketplace,0.493735
unknown,0.524231
exchange,0.499174
illicit,0.496114
individual,0.499417


Mixer Usuage Rate

In [0]:
%sql
SELECT is_mixer, COUNT(*) AS count, ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS pct
FROM simulated_blockchain_transactions
GROUP BY is_mixer;

is_mixer,count,pct
true,987,4.94
false,19013,95.07


Illict VS Licit

In [0]:
%sql
SELECT label, COUNT(*) AS txn_count
FROM simulated_blockchain_transactions
GROUP BY label;

label,txn_count
licit,17122
illicit,2878


Illict Activity By Entity Type

In [0]:
%sql
SELECT entity_type, COUNT(*) AS illicit_txns
FROM simulated_blockchain_transactions
WHERE label = 'illicit'
GROUP BY entity_type
ORDER BY illicit_txns DESC;

entity_type,illicit_txns
illicit,1981
individual,532
exchange,221
marketplace,92
unknown,52


Most Active Clusters

In [0]:
%sql
SELECT cluster_id, COUNT(*) AS txn_count
FROM simulated_blockchain_transactions
WHERE cluster_id IS NOT NULL
GROUP BY cluster_id
ORDER BY txn_count DESC
LIMIT 10;

cluster_id,txn_count
cluster_123,108
cluster_152,107
cluster_198,104
cluster_194,103
cluster_125,103
cluster_191,102
cluster_187,101
cluster_129,100
cluster_159,99
cluster_175,97


Clusters with most illict activity

In [0]:
%sql
SELECT 
  cluster_id,
  COUNT(*) AS total_txns,
  SUM(CASE WHEN label = 'illicit' THEN 1 ELSE 0 END) AS illicit_txns,
  ROUND(SUM(CASE WHEN label = 'illicit' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS pct_illicit
FROM simulated_blockchain_transactions
WHERE cluster_id IS NOT NULL
GROUP BY cluster_id
HAVING COUNT(*) >= 10
ORDER BY pct_illicit DESC
LIMIT 10;

cluster_id,total_txns,illicit_txns,pct_illicit
cluster_120,26,26,100.00
cluster_161,18,18,100.00
cluster_151,86,65,75.58
cluster_72,41,29,70.73
cluster_41,57,39,68.42
cluster_173,42,26,61.90
cluster_92,46,27,58.70
cluster_154,33,18,54.55
cluster_100,41,22,53.66
cluster_189,38,20,52.63


Daily Transaction Volume

In [0]:
%sql
SELECT to_date(timestamp) AS day, COUNT(*) AS txn_count
FROM simulated_blockchain_transactions
GROUP BY day
ORDER BY day;

day,txn_count
null,20000


Illict Activity Over Time

In [0]:
%sql
SELECT date_trunc('DAY', timestamp) AS day,
       SUM(CASE WHEN label = 'illicit' THEN 1 ELSE 0 END) AS illicit_txns,
       COUNT(*) AS total_txns,
       ROUND(SUM(CASE WHEN label = 'illicit' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS pct_illicit
FROM simulated_blockchain_transactions
GROUP BY day
ORDER BY day;

day,illicit_txns,total_txns,pct_illicit
null,2878,20000,14.39
